In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1.1


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss(num_neg=4,margin=0.1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.MatchPyramid.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8903.23it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5860.61it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3532316.96it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10655.63it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5892.89it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1053496.53it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10698.19it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7ff616b77350>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7ff60fe35a50>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    batch_size=100,
    resample=True,
    sort=False,
    shuffle=True
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=160,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.MatchPyramid.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.MatchPyramid()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['dropout_rate'] = 0.2
model.params['mask_value'] = 0

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchPyramid(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (matching): Matching()
  (conv2d): Sequential(
    (0): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(1, 16, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(16, 32, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
  )
  (dpool_layer): AdaptiveAvgPool2d(output_size=[3, 10])
  (dropout): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=960, out_features=1, bias=True)
)
Trainable params:  9023161


In [11]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5
)

In [12]:
trainer.run()

[Iter-11 Loss-0.089]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4649 - normalized_discounted_cumulative_gain@5(0.0): 0.5368 - mean_average_precision(0.0): 0.4976



[Iter-22 Loss-0.075]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4942 - normalized_discounted_cumulative_gain@5(0.0): 0.5652 - mean_average_precision(0.0): 0.5236



[Iter-33 Loss-0.059]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5387 - normalized_discounted_cumulative_gain@5(0.0): 0.5963 - mean_average_precision(0.0): 0.5565



[Iter-44 Loss-0.038]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5638 - normalized_discounted_cumulative_gain@5(0.0): 0.6087 - mean_average_precision(0.0): 0.5693



[Iter-55 Loss-0.022]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.567 - normalized_discounted_cumulative_gain@5(0.0): 0.6239 - mean_average_precision(0.0): 0.5712

Cost time: 41.48978304862976s
